# Imports

In [1]:
print('Running...')

from pathlib import Path
import os
HERE = Path(os.path.abspath(''))
os.chdir(HERE.parent)

# config imports
from cfg import config

# custom beacon imports
from analysis.reader import Reader
from analysis.sinesubtraction import SineSubtract
from analysis import reconstruction
from analysis import utils

# standard imports
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import savgol_filter
import pandas as pd
from glob import glob

# import for interactive viewing
from tqdm.auto import tqdm
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

os.chdir(HERE)

print('Finished.')

Running...
Welcome to JupyROOT 6.28/06
Finished.


# Test Config

In [ ]:
print(f'Datapath: {config.BEACON_DATAPATH}')
print(f'Runs: Length = {len(config.RUNS)}, Last = {config.RUNS[-1]}')
print(f'Sample Rate: {config.SAMPLE_RATE} S/s')
print(f'ADC Range: {config.ADC_RANGE} ADU')
print(f'Sample Size: {config.SAMPLE_SIZE}')
print(f'Scint Mapping: {config.SCINT_MAP}')
print(f'HPOL Mapping: {config.HPOL_MAP}')
print(f'VPOL Mapping: {config.VPOL_MAP}')
print(f'HPOL Positions: {config.RF_HPOL_POS}')
print(f'HPOL Delays: {config.RF_HPOL_DELAYS}')
print(f'VPOL Positions: {config.RF_VPOL_POS}')
print(f'VPOL Delays: {config.RF_VPOL_DELAYS}')
print(f'Scint Corner Positions: {config.SCINT_POS_CORNERS}')
print(f'Scint Center Positions: {config.SCINT_POS}')
print(f'Scint Delays: {config.SCINT_DELAYS}')
print(f'Scint N-S Tilts: {config.SCINT_TILTS_NS}')
print(f'Scint E-W Tilts: {config.SCINT_TILTS_EW}')
print(f'Scint Dimensions: {config.SCINT_DIMENSIONS}')
print(f'Scint Face Normal Vectors: {config.SCINT_FACE_NORMALS}')
print(f'RF Reconstruction Required Number of Antennas: {config.RF_RECON_REQUISITE}')
print(f'Scint Reconstruction Required Number of Scintillators: {config.SCINT_RECON_REQUISITE}')
print(f'Scint Amplitude Exclusion Threshold for Reconstruction: {config.SCINT_EXCLUSION_THRESHOLD}')

# Test Reader

In [16]:
run = widgets.IntText(
        value=0,
        description='Run: ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
    )
entry = widgets.IntText(
        value=0,
        description='Entry: ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
    )
trigType = widgets.Dropdown(
        options={None:None, 'SW':1,'COIN': 4},
        value=None,
        description='Trig Type: ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
    )

@interact(run=run, entry=entry, trigType=trigType)
def runWidget(run, entry, trigType):
    reader = Reader(config.BEACON_DATAPATH, run)

    event = entry
    if not (trigType is None):
        trigger_types = reader.loadTriggerTypes()
        eventids = np.where(trigger_types == trigType)[0]
        event = eventids[entry]
    
    reader.setEntry(event) # sets reader to a specific event
    
    fig, axes = plt.subplots(ncols=4, nrows=4, figsize=[20,10])
    fig.suptitle(f'Run {run} Event {event}')
    fig.supxlabel('Time [ns]'); fig.supylabel('Voltage [ADU]')
    for ch, ax in zip(list(range(16)), axes.ravel()):
        ax.plot(reader.t(), reader.wf(ch))
    fig.tight_layout(pad=1)

interactive(children=(IntText(value=0, description='Run: '), IntText(value=0, description='Entry: '), Dropdown…

# Test Reconstruction

In [2]:
# RF
CWsub = SineSubtract(0, 0.125, 0.03)
reader = Reader(config.BEACON_DATAPATH, 251)

entry = widgets.SelectionSlider(
        options=np.arange(len(reader.loadTriggerTypes())),
        value=0,
        description='Entry:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
    )
@interact(entry=entry)
def run(entry):
# entry=6
    corr = reconstruction.mapRF(reader, entry, 'h', CWsub=CWsub, resolution=0.25, upsample_factor=100, plot=True, verbose=1, mask=9)[0]#True)
    print(f'Score: {np.max(corr)}')

display()

interactive(children=(SelectionSlider(description='Entry:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,…

In [6]:
# scint
reader = Reader(config.BEACON_DATAPATH, 270)
corr,_,_,arrTimes = reconstruction.mapScint(reader, 5670, resolution=0.25, plot=1, verbose=1, plot_contours=1)#[1,1,0])
print(f'Returned Arrival Times: {arrTimes}')
print(f'Score: {np.max(corr)}')

if ~np.isnan(corr):
    # quantify spread; akin to FWHM
    phiMap = np.sum(corr, axis=0)
    thetaMap = np.sum(corr, axis=1)
    phiSpread = len(phiMap[phiMap >= 0.5*np.max(phiMap)])*0.25
    thetaSpread = len(thetaMap[thetaMap >= 0.5*np.max(thetaMap)])*0.25
    """
    Zenith Spread : real number
        A quantification for the 'spread' of the probability map, using the total size of FWHM's of each peak in Zenith.
    Azimuth Spread : real number
        A quantification for the 'spread' of the probability map, using the total size of FWHM's of each peak in Azimuth.
    """
    # 1d distributions and spread
    phi = np.arange(-180, 180+0.25/2, 0.25)
    theta = np.arange(0, 180+0.25/2, 0.25)
    fspread,aspread = plt.subplots(ncols=2,figsize=[15,2])
    aspread[0].plot(theta, thetaMap)
    aspread[0].plot(theta[thetaMap >= 0.5*np.max(thetaMap)], thetaMap[thetaMap >= 0.5*np.max(thetaMap)], c='red')
    aspread[0].axhline(0.5*np.max(thetaMap), ls='--', alpha=0.5, color='gray')
    aspread[0].set(ylabel='Density', xlabel='Zenith')
    aspread[1].plot(phi, phiMap)
    aspread[1].plot(phi[phiMap >= 0.5*np.max(phiMap)], phiMap[phiMap >= 0.5*np.max(phiMap)], c='red')
    aspread[1].axhline(0.5*np.max(phiMap), ls='--', alpha=0.5, color='gray')
    aspread[1].set(ylabel='Density', xlabel='Azimuth')
    print(f'Zenith, Azimuth "Spread" = {thetaSpread}, {phiSpread}')

Too many non-zero scint channels. See "SCINT_RECON_REQUISITE" and "SCINT_EXCLUSION_THRESHOLD" in cfg.config.
Returned Arrival Times: nan
Score: nan


# Test Scint Peak, Risetime, and Pulse ID

In [20]:
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
from matplotlib.offsetbox import AnchoredText


runn = 501
reader = Reader(config.BEACON_DATAPATH, runn)
scintEntries = np.where(reader.loadTriggerTypes()==4)[0]

entry = widgets.SelectionSlider(
        options=scintEntries,
        value=scintEntries[0],
        description='Scint Entry:',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
    )

t = reader.t()

@interact(entry=entry)
def run(entry):
    reader.setEntry(entry)
    fig, axes = plt.subplots(nrows=4, figsize=[30,15])
    fig.suptitle(f'Run {runn} Event {entry}', fontsize=20)
    fig.supxlabel('Time [ns]', fontsize=20)
    fig.supylabel('Voltage [ADU]', fontsize=20)
    
    for j in range(4):
        wfm = reader.wf(j)
        ax = axes[j]
    
        inds, amps = utils.getPeaks(wfm)
        rt, drt = utils.getRisetimes(t, wfm)
        pulseInts, pulseInds, totalInt, posInt = utils.integratePulses(t, wfm)
        
        ax.plot(t, wfm)
        ax.fill_between(t[wfm>=0], 0, wfm[wfm>=0], color='red', alpha=0.1)
        
        for num in range(len(inds)):
            i = inds[num]
            ax.scatter(t[inds[num]], amps[num], s=50, c='black')
        
            ax.plot(t[pulseInds[num]], wfm[pulseInds[num]], c='red')
            # ax.fill_between(t[pulseInds[num]], wfm[pulseInds[num]], color='red', alpha=0.1)
    
            co = 'blue' if num == 0 else 'black'
        
            ymin, ymax = ax.get_ylim()
            yrange = ymax - ymin
            v = lambda y: (y-ymin)/(ymax-ymin)
            vmin = v(amps[num]/2-0.4*yrange)
            vmax = v(amps[num]/2+0.4*yrange)
            ax.axvline(rt[num], ymin=0, ymax=1, color=co, ls=':', lw=3)
            
            ax.errorbar(rt[num], amps[num]/2, xerr=drt[num], capsize=6, color=co, lw=3, capthick=3)
    
            # ax.add_artist(AnchoredText(f'Amplitude: {amps[num]:0.0f} ADU',
            #                            loc="lower left", frameon=True, prop=dict(fontsize=20)))
            # ax.add_artist(AnchoredText(f'Amplitudes: {", ".join(["{0:0.2f}".format(am) for am in amps])} ADU', loc="lower left", frameon=True, prop=dict(fontsize=30)))
            ax.add_artist(AnchoredText(f'Positive Integration: {posInt:0.0f}', bbox_to_anchor=(0, 0.2), bbox_transform=ax.transAxes,
                                       loc="lower left", frameon=True, prop=dict(fontsize=20)))
            ax.add_artist(AnchoredText(f'Risetimes: {", ".join(["{0:0.2f}+/-{1:0.2f}".format(r, dr) for r, dr in zip(rt, drt)])}', loc="lower left", frameon=True, prop=dict(fontsize=20)))
    
    fig.tight_layout(pad=2)

interactive(children=(SelectionSlider(description='Scint Entry:', options=(1, 3, 5, 6, 9, 12, 13, 16, 18, 20, …